In [1]:
cd ..

C:\Users\User\Documents\SURF 2021\RL RNN SURF\src


We first define the input data. The input data is has the following structure: We have a group of subjects (say S1 and S2) and each subject has completed the decision-making task multiple times (multiple blocks), say each subject has completed the task two times in this example, i.e., we have two blocks of data for each subject. The data within each block is a ditonary containing three numpy arrays: 'action', 'state', 'reward'. 

'action' containig the actions taken by the subject on each trial and it should be a non-zero integer or -1. If the action is -1 it will coded by a zero vector and corresponds to no-action. Dimesionlity of 'action' is B x |T| in which |T| is the number of trials.

'state' contains the state of the environment each trial. Its dimesionlity is B x |T| x |S| in which |T| is the number of trials, and |S| is the lenght of state vector. 

'reward' contains the reward received after taking each actions. Its dimesionlity is B x |T| in which |T| is the number of trials.

For example, if subject S1 has completed 6 trials in the firt block and 4 trials in the second block and subject 2 has completed 5, 6 trials in the first and second blocks respectivly, then the data structure can look like this:

   

In [2]:
#  Data
import numpy as np
import csv
data={}
test_data = {}
i=0
curr = ''
with open('quartileData\\taskData_bdiBottom.csv', 'r' ) as theFile:
    reader = csv.reader(theFile)
    headers = next(reader, None)
    for line in reader:
        if line[70] != curr:
            if i > 0:
                block = [
                    {
                        'action': np.array([actions])-1,
                        'state': np.array([states]),
                        'reward': np.array([rewards]),
                        'id': curr,

                        'block': 0
                    }

                ]
#                 if i<226:
                data[curr] = block
#                 else:
                test_data[curr] = block
            curr = line[70]
            i += 1
            actions = []
            states = []
            rewards = []
        actions.append(int(float(line[8])))
        states.append([int(float(line[3])),int(float(line[4]))])
        rewards.append(int(float(line[10])))

In [3]:
data

{'A10FTGEV1WR18H': [{'action': array([[1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1,
           1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1,
           1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,
           0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1,
           1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0,
           0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
           0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
           1, 0, 0, 1, 1, 1]]),
   'state': array([[[ 2,  4],
           [ 2,  4],
           [ 2,  4],
           [ 3,  4],
           [ 2,  4],
           [ 4,  3],
           [ 4,  3],
           [ 4,  2],
           [ 2,  3],
           [ 2,  3],
           [ 4,  2],
           [ 4,  3],
           [ 2,  4],
           [ 4,  2],
           [ 2,  3],
           [ 4,  2],
           [ 4, 10],
           [10,  4],
           [10

In the above example, |A|=2 (there are two actions coded as 0 and 1), and |S|=2 (the state vector has two elements). For example, if there are three stimuli in the environment, they can be coded as [1, 0, 0], [0, 1, 0], [0, 0, 1] state vectors. In this case |S|=3.

In [4]:
from actionflow.rnn_noAction.lstm_beh import LSTMBeh
worker = LSTMBeh(a_size=2, s_size=2, n_cells=20)

c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\user\documents\surf

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


Finally, we train the model:

In [5]:
from actionflow.rnn_noAction.opt_beh import OptBEH
from actionflow.util.logger import LogFile

output_path = '../results/taskData_bdiBottom_noAction/'
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, None,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None
                    )

2021-08-12 14:18:45,078 - DeepRL - DEBUG - version control: (None, None)
2021-08-12 14:18:45,078 - DeepRL - DEBUG - learning rate: 0.01
2021-08-12 14:18:45,078 - DeepRL - DEBUG - global iters: 50
2021-08-12 14:18:45,078 - DeepRL - DEBUG - training data-points: 262
2021-08-12 14:18:45,087 - DeepRL - DEBUG - test data-points: None


Instructions for updating:
Use tf.cast instead.


2021-08-12 14:18:48,238 - DeepRL - DEBUG - opt started...


INFO:tensorflow:../results/taskData_bdiBottom_noAction/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-08-12 14:19:21,759 - DeepRL - DEBUG - global iter =    0 total obj: 28436.1334
2021-08-12 14:19:54,595 - DeepRL - DEBUG - global iter =    1 total obj: 28411.0551
2021-08-12 14:20:31,847 - DeepRL - DEBUG - global iter =    2 total obj: 28395.3228
2021-08-12 14:21:03,729 - DeepRL - DEBUG - global iter =    3 total obj: 28389.8524
2021-08-12 14:21:38,202 - DeepRL - DEBUG - global iter =    4 total obj: 28385.6464
2021-08-12 14:22:17,688 - DeepRL - DEBUG - global iter =    5 total obj: 28381.9398
2021-08-12 14:22:56,350 - DeepRL - DEBUG - global iter =    6 total obj: 28378.5359
2021-08-12 14:23:36,430 - DeepRL - DEBUG - global iter =    7 total obj: 28374.3867
2021-08-12 14:24:15,258 - DeepRL - DEBUG - global iter =    8 total obj: 28370.1361
2021-08-12 14:24:53,912 - DeepRL - DEBUG - global iter =    9 total obj: 28365.0960
2021-08-12 14:25:33,255 - DeepRL - DEBUG - global iter =   10 total obj: 28362.7853
2021-08-12 14:26:12,064 - DeepRL - DEBUG - global iter =   11 total obj: 283

INFO:tensorflow:../results/taskData_bdiBottom_noAction/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [6]:
# from actionflow.data.data_process import DataProcess
# train_merged = DataProcess.merge_data(data)
# train_merged

And then the merged data can be used for training the model as before. The test data can also be passed to the training method, in order to test the mmodel on the training data in regular intervals. Say the test data is as follows:

and we want to test the model every 10 iterations:

In [7]:
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, test_data,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None,
                    test_period=10
                    )

2021-08-12 14:47:13,413 - DeepRL - DEBUG - version control: (None, None)
2021-08-12 14:47:13,415 - DeepRL - DEBUG - learning rate: 0.01
2021-08-12 14:47:13,417 - DeepRL - DEBUG - global iters: 50
2021-08-12 14:47:13,418 - DeepRL - DEBUG - training data-points: 262
2021-08-12 14:47:13,419 - DeepRL - DEBUG - test data-points: 262
2021-08-12 14:47:16,804 - DeepRL - DEBUG - opt started...
2021-08-12 14:47:16,806 - DeepRL - DEBUG - started testing...
2021-08-12 14:50:20,525 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/taskData_bdiBottom_noAction/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-08-12 14:51:01,001 - DeepRL - DEBUG - global iter =    0 total obj: 28468.9125
2021-08-12 14:51:28,841 - DeepRL - DEBUG - global iter =    1 total obj: 28415.4751
2021-08-12 14:51:55,987 - DeepRL - DEBUG - global iter =    2 total obj: 28412.7130
2021-08-12 14:52:17,950 - DeepRL - DEBUG - global iter =    3 total obj: 28411.4227
2021-08-12 14:52:39,624 - DeepRL - DEBUG - global iter =    4 total obj: 28409.9606
2021-08-12 14:53:01,328 - DeepRL - DEBUG - global iter =    5 total obj: 28408.4902
2021-08-12 14:53:24,401 - DeepRL - DEBUG - global iter =    6 total obj: 28407.4510
2021-08-12 14:53:46,474 - DeepRL - DEBUG - global iter =    7 total obj: 28403.8802
2021-08-12 14:54:08,322 - DeepRL - DEBUG - global iter =    8 total obj: 28395.9987
2021-08-12 14:54:30,126 - DeepRL - DEBUG - global iter =    9 total obj: 28392.1729
2021-08-12 14:54:30,127 - DeepRL - DEBUG - started testing...
2021-08-12 14:58:26,498 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/taskData_bdiBottom_noAction/model-10/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-08-12 14:59:26,027 - DeepRL - DEBUG - global iter =   10 total obj: 28384.5608
2021-08-12 14:59:49,331 - DeepRL - DEBUG - global iter =   11 total obj: 28377.8537
2021-08-12 15:00:11,931 - DeepRL - DEBUG - global iter =   12 total obj: 28373.8331
2021-08-12 15:00:34,187 - DeepRL - DEBUG - global iter =   13 total obj: 28368.8869
2021-08-12 15:00:56,378 - DeepRL - DEBUG - global iter =   14 total obj: 28361.1411
2021-08-12 15:01:18,180 - DeepRL - DEBUG - global iter =   15 total obj: 28357.1722
2021-08-12 15:01:39,260 - DeepRL - DEBUG - global iter =   16 total obj: 28353.8564
2021-08-12 15:01:58,257 - DeepRL - DEBUG - global iter =   17 total obj: 28348.3697
2021-08-12 15:02:17,303 - DeepRL - DEBUG - global iter =   18 total obj: 28344.2073
2021-08-12 15:02:36,325 - DeepRL - DEBUG - global iter =   19 total obj: 28340.3964
2021-08-12 15:02:36,326 - DeepRL - DEBUG - started testing...
2021-08-12 15:08:07,082 - DeepRL - DEBUG - finished testing.


...

INFO:tensorflow:../results/taskData_bdiBottom_noAction/model-30/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-08-12 15:17:54,275 - DeepRL - DEBUG - global iter =   30 total obj: 28261.8586
2021-08-12 15:18:13,230 - DeepRL - DEBUG - global iter =   31 total obj: 28252.2148
2021-08-12 15:18:32,148 - DeepRL - DEBUG - global iter =   32 total obj: 28242.8047
2021-08-12 15:18:51,049 - DeepRL - DEBUG - global iter =   33 total obj: 28234.1494
2021-08-12 15:19:10,642 - DeepRL - DEBUG - global iter =   34 total obj: 28228.8174
2021-08-12 15:19:32,541 - DeepRL - DEBUG - global iter =   35 total obj: 28215.0867
2021-08-12 15:19:54,483 - DeepRL - DEBUG - global iter =   36 total obj: 28215.9082
2021-08-12 15:20:16,252 - DeepRL - DEBUG - global iter =   37 total obj: 28217.0860
2021-08-12 15:20:37,907 - DeepRL - DEBUG - global iter =   38 total obj: 28193.2393
2021-08-12 15:20:59,670 - DeepRL - DEBUG - global iter =   39 total obj: 28175.3598
2021-08-12 15:20:59,671 - DeepRL - DEBUG - started testing...
2021-08-12 15:26:31,072 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/taskData_bdiBottom_noAction/model-40/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-08-12 15:27:01,491 - DeepRL - DEBUG - global iter =   40 total obj: 28162.1247
2021-08-12 15:27:20,449 - DeepRL - DEBUG - global iter =   41 total obj: 28195.8174
2021-08-12 15:27:39,680 - DeepRL - DEBUG - global iter =   42 total obj: 28158.5222
2021-08-12 15:27:58,791 - DeepRL - DEBUG - global iter =   43 total obj: 28134.7396
2021-08-12 15:28:19,768 - DeepRL - DEBUG - global iter =   44 total obj: 28121.5076
2021-08-12 15:28:41,830 - DeepRL - DEBUG - global iter =   45 total obj: 28108.8598
2021-08-12 15:29:02,257 - DeepRL - DEBUG - global iter =   46 total obj: 28107.5097
2021-08-12 15:29:20,124 - DeepRL - DEBUG - global iter =   47 total obj: 28090.9980
2021-08-12 15:29:38,117 - DeepRL - DEBUG - global iter =   48 total obj: 28075.4882
2021-08-12 15:29:56,116 - DeepRL - DEBUG - global iter =   49 total obj: 28062.6540
2021-08-12 15:29:56,117 - DeepRL - DEBUG - opt finished.
2021-08-12 15:29:56,118 - DeepRL - DEBUG - started testing...
2021-08-12 15:32:30,226 - DeepRL - DEBUG 

INFO:tensorflow:../results/taskData_bdiBottom_noAction/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [8]:
# test_data["S248"]